# Внутренняя калибровка камер

In [ ]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2 as cv
import glob
import pandas as pd
from google.colab.patches import cv2_imshow
import subprocess
import json
from moviepy.editor import VideoFileClip
import cv2
import os

In [ ]:
#@title Информация о видео
# Исходяшее видео
video_1 = '/content/drive/My Drive/video/cam1.mp4'
video_2 = '/content/drive/My Drive/video/cam2.mp4'
video_3 = '/content/drive/My Drive/video/cam3.mp4'
video_4 = '/content/drive/My Drive/video/cam4.mp4'
video_list =[video_1, video_2, video_3, video_4]


# Функция для получения информации о кодеке с помощью ffprobe
def get_video_codec(video_path):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=codec_name',
        '-of', 'json',
        video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    info = json.loads(result.stdout)
    codec = info['streams'][0]['codec_name'] if 'streams' in info and len(info['streams']) > 0 else 'Unknown'
    return codec

# Функция для вывода информации о видео
def print_video_info(video_paths):
    for video_path in video_paths:
        with VideoFileClip(video_path) as video:
            codec = get_video_codec(video_path)
            duration = video.duration
            fps = video.fps
            frame_count = int(fps * duration)  # Количество кадров

            print(f"Video: {video_path}")
            print(f"  Duration: {duration} seconds")
            print(f"  FPS: {fps}")
            print(f"  Frame Count: {frame_count}")  # Вывод количества кадров
            print(f"  Resolution: {video.size[0]}x{video.size[1]}")
            print(f"  Codec: {codec}")
            print()
# Вызов функции
print_video_info(video_list)

  if event.key is 'enter':



Video: /content/drive/My Drive/video/cam1.mp4
  Duration: 1179.32 seconds
  FPS: 30.0
  Frame Count: 35379
  Resolution: 2224x1080
  Codec: h264

Video: /content/drive/My Drive/video/cam2.mp4
  Duration: 1234.86 seconds
  FPS: 29.83
  Frame Count: 36835
  Resolution: 1920x1080
  Codec: h264

Video: /content/drive/My Drive/video/cam3.mp4
  Duration: 1289.53 seconds
  FPS: 30.0
  Frame Count: 38685
  Resolution: 1920x1080
  Codec: h264

Video: /content/drive/My Drive/video/cam4.mp4
  Duration: 1338.49 seconds
  FPS: 30.0
  Frame Count: 40154
  Resolution: 1920x1080
  Codec: h264




```
,Доска на камере,,"Длительность, сек"
cam_01,63,78,15
cam_02,86,99,13
cam_03,109,126,17
cam_04,135,148,13
```


In [ ]:
#@title Извлечение кадров из видео около 100 кадров....

# Путь к папкам с изображениями на Google Диске
path_image_1 = "/content/drive/My Drive/image/chessboard/cam1/"
path_image_2 = "/content/drive/My Drive/image/chessboard/cam2/"
path_image_3 = "/content/drive/My Drive/image/chessboard/cam3/"
path_image_4 = "/content/drive/My Drive/image/chessboard/cam4/"

# Создаем папки, если они не существуют
os.makedirs(path_image_1, exist_ok=True)
os.makedirs(path_image_2, exist_ok=True)
os.makedirs(path_image_3, exist_ok=True)
os.makedirs(path_image_4, exist_ok=True)

# Путь к видео
video_paths = {
    'cam1.mp4': '/content/drive/MyDrive/video/cam1.mp4',
    'cam2.mp4': '/content/drive/MyDrive/video/cam2.mp4',
    'cam3.mp4': '/content/drive/MyDrive/video/cam3.mp4',
    'cam4.mp4': '/content/drive/MyDrive/video/cam4.mp4'
}

# Время начала и конца в секундах
time_intervals = {
    'cam1.mp4': (63, 78),
    'cam2.mp4': (86, 99),
    'cam3.mp4': (109, 126),
    'cam4.mp4': (135, 148)
}

# Функция для подсчета кадров
def count_frames(video_path, start_time, end_time):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print(f"Ошибка: Не удалось получить FPS для {video_path}.")
        return 0
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    cap.release()
    return end_frame - start_frame

# Функция для получения длительности видео
def get_video_duration(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps if fps > 0 else 0
    cap.release()
    return duration

# Функция для извлечения и сохранения кадров
def save_frames(video_path, start_time, end_time, save_path, target_count=99):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame

    if total_frames <= 0:
        print(f"Ошибка: Общее количество кадров в интервале для {video_path} равно нулю.")
        return

    # Рассчитываем шаг для выборки кадров
    step = max(1, total_frames // target_count)  # минимум 1 кадр

    current_frame = 0
    saved_frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if start_frame <= current_frame < end_frame and (current_frame - start_frame) % step == 0:
            frame_filename = os.path.join(save_path, f"frame_{saved_frame_count:04d}.jpg")
            if not cv2.imwrite(frame_filename, frame):
                print(f"Ошибка: Не удалось сохранить кадр {frame_filename}.")
            else:
                saved_frame_count += 1
        current_frame += 1
        # Прерываем, если достигли конца видео
        if current_frame >= end_frame:
            break
    cap.release()
    print(f"Сохранено {saved_frame_count} кадров в {save_path}")

# Подсчет кадров и сохранение для каждого видео
for video_name, video_path in video_paths.items():
    if not os.path.exists(video_path):
        print(f"Ошибка: Файл {video_path} не найден.")
        continue

    duration = get_video_duration(video_path)
    start_time, end_time = time_intervals[video_name]

    if start_time >= duration or end_time > duration:
        print(f"Ошибка: Временной интервал для {video_name} выходит за пределы длительности видео ({duration:.2f} секунд).")
        continue

    frame_count = count_frames(video_path, start_time, end_time)
    print(f"{video_name}: {frame_count} кадров")

    # Определяем путь для сохранения кадров
    if 'cam1' in video_name:
        save_frames(video_path, start_time, end_time, path_image_1)
    elif 'cam2' in video_name:
        save_frames(video_path, start_time, end_time, path_image_2)
    elif 'cam3' in video_name:
        save_frames(video_path, start_time, end_time, path_image_3)
    elif 'cam4' in video_name:
        save_frames(video_path, start_time, end_time, path_image_4)

cam1.mp4: 450 кадров
Сохранено 113 кадров в /content/drive/My Drive/image/chessboard/cam1/
cam2.mp4: 387 кадров
Сохранено 129 кадров в /content/drive/My Drive/image/chessboard/cam2/
cam3.mp4: 510 кадров
Сохранено 102 кадров в /content/drive/My Drive/image/chessboard/cam3/
cam4.mp4: 386 кадров
Сохранено 129 кадров в /content/drive/My Drive/image/chessboard/cam4/


In [ ]:
#@title Ищем шашечки на 4-й камере

# Папка с изображениями
image_folder = '/content/drive/MyDrive/image/chessboard/cam4/*'
images = glob.glob(image_folder)

# Диапазон для поиска размера шахматной доски
min_squares = 3
max_squares = 6
min_squares_х = 3
max_squares_х = 3
# Проверка количества изображений
print(f"Найдено изображений: {len(images)}")

# Создаем список для хранения результатов
results = []

# Перебираем каждое изображение
for fname in images:
    img = cv.imread(fname)
    if img is None:
        print(f"Не удалось загрузить изображение: {fname}")
        continue

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (5, 5), 0)

    # Инициализируем переменные для хранения лучших параметров
    best_size = None
    max_found_x = 0
    max_found_y = 0

    # Перебираем все возможные размеры шахматной доски
    for num_squares_x in range(min_squares_х, max_squares_х + 1):
        for num_squares_y in range(min_squares, max_squares + 1):
            ret, corners = cv.findChessboardCorners(gray, (num_squares_x, num_squares_y), None)
            if ret:
                # Если текущий размер больше, чем уже найденный, обновляем лучший размер
                if (num_squares_x * num_squares_y) > (max_found_x * max_found_y):
                    max_found_x = num_squares_x
                    max_found_y = num_squares_y
                    best_size = (num_squares_x, num_squares_y)

    # Сохраняем результаты
    results.append({
        'image': fname,
        'best_size': best_size if best_size else "Not found",
        'found': True if best_size else False
    })

    if best_size:
        print(f"Найдена шахматная доска в изображении: {fname} с максимальными размерами {best_size[0]}x{best_size[1]}")
    else:
        print(f"Шахматная доска не найдена в изображении: {fname}")

# Создаем DataFrame из результатов
df = pd.DataFrame(results)

# Отображаем таблицу результатов
print(df)

Найдено изображений: 387
Не удалось загрузить изображение: /content/drive/MyDrive/image/chessboard/cam4/chessboard_cam4.xlsx
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0000.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0001.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0002.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0003.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0004.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0005.jpg с максимальными размерами 3x4
Найдена шахматная доска в изображении: /content/drive/MyDrive/image/chessboard/cam4/frame_0006.jpg с 

In [ ]:
df

,image,best_size,found
0,/content/drive/MyDrive/image/chessboard/cam4/f...,"(3, 4)",True
1,/content/drive/MyDrive/image/chessboard/cam4/f...,"(3, 4)",True
2,/content/drive/MyDrive/image/chessboard/cam4/f...,"(3, 4)",True
3,/content/drive/MyDrive/image/chessboard/cam4/f...,"(3, 4)",True
4,/content/drive/MyDrive/image/chessboard/cam4/f...,"(3, 4)",True
...,...,...,...
381,/content/drive/MyDrive/image/chessboard/cam4/f...,Not found,False
382,/content/drive/MyDrive/image/chessboard/cam4/f...,Not found,False
383,/content/drive/MyDrive/image/chessboard/cam4/f...,Not found,False
384,/content/drive/MyDrive/image/chessboard/cam4/f...,Not found,False


In [ ]:
#@title Вариант 2 - все  изображениz
# Установка необходимых библиотек
!pip install toml opencv-python-headless

# Импорт библиотек
import os
import glob
import logging
import numpy as np
import cv2
import re
import toml
from google.colab import drive
from google.colab.patches import cv2_imshow  # Для отображения изображений в Colab

# Подключение к Google Drive
drive.mount('/content/drive')

# Настройка логирования
logging.basicConfig(level=logging.INFO)

def findCorners(img_path, corner_nb, objp=[], show=True):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    img = cv2.imread(img_path)
    if img is None:
        cap = cv2.VideoCapture(img_path)
        ret, img = cap.read()
        if not ret:
            logging.error(f"Failed to read image or video frame from {img_path}.")
            return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    ret, corners = cv2.findChessboardCorners(gray, corner_nb, None)

    if ret:
        imgp = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        logging.info(f'{os.path.basename(img_path)}: Corners found.')
        if show:
            cv2.drawChessboardCorners(img, corner_nb, imgp, ret)
            for i, corner in enumerate(imgp):
                if i in [0, corner_nb[0] - 1, corner_nb[0] * (corner_nb[1] - 1), corner_nb[0] * corner_nb[1] - 1]:
                    x, y = corner.ravel()
                    cv2.putText(img, str(i + 1), (int(x) - 5, int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 7)
                    cv2.putText(img, str(i + 1), (int(x) - 5, int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            print('Шахматка найдена', img_path)
        return imgp
    else:
        if show:
            logging.info(f'{os.path.basename(img_path)}: Corners not found: please label them by hand.')
            print('Шахматка не найдена', img_path)
        return None

def extract_frames(video_path, extract_every_N_sec, overwrite_extraction):
    # Здесь должна быть реализация извлечения кадров из видео
    # Это примерная заготовка функции
    pass

def calib_calc_fun(calib_dir, intrinsics_config_dict):
    overwrite_intrinsics = intrinsics_config_dict.get('overwrite_intrinsics')
    try:
        calib_file = glob.glob(os.path.join(calib_dir, f'Calib*.toml'))[0]
    except IndexError:
        calib_file = None

    if not overwrite_intrinsics and calib_file:
        logging.info(f'\nPreexisting calibration file found: \'{calib_file}\'.')
        logging.info(f'\nRetrieving intrinsic parameters from file. Set "overwrite_intrinsics" to true in Config.toml to recalculate them.')
        calib_data = toml.load(calib_file)
        ret, C, S, D, K, R, T = [], [], [], [], [], [], []
        for cam in calib_data:
            if cam != 'metadata':
                ret += [0.0]
                C += [calib_data[cam]['name']]
                S += [calib_data[cam]['size']]
                K += [np.array(calib_data[cam]['matrix'])]
                D += [calib_data[cam]['distortions']]
                R += [[0.0, 0.0, 0.0]]
                T += [[0.0, 0.0, 0.0]]
        nb_cams_intrinsics = len(C)
    else:
        logging.info(f'\nCalculating intrinsic parameters...')
        ret, C, S, D, K, R, T = calibrate_intrinsics(calib_dir, intrinsics_config_dict)
        nb_cams_intrinsics = len(C)
    return ret, C, S, D, K, R, T

def calibrate_intrinsics(calib_dir, intrinsics_config_dict):
    try:
        intrinsics_cam_listdirs_names = next(os.walk(os.path.join(calib_dir, 'intrinsics')))[1]
    except StopIteration:
        logging.exception(f'Error: No {os.path.join(calib_dir, "intrinsics")} folder found.')
        raise Exception(f'Error: No {os.path.join(calib_dir, "intrinsics")} folder found.')

    intrinsics_extension = intrinsics_config_dict.get('intrinsics_extension')
    extract_every_N_sec = intrinsics_config_dict.get('extract_every_N_sec')
    overwrite_extraction = False
    show_detection_intrinsics = intrinsics_config_dict.get('show_detection_intrinsics')
    intrinsics_corners_nb = intrinsics_config_dict.get('intrinsics_corners_nb')
    intrinsics_square_size = intrinsics_config_dict.get('intrinsics_square_size') / 1000

    ret, C, S, D, K, R, T = [], [], [], [], [], [], []

    for i, cam in enumerate(intrinsics_cam_listdirs_names):
        objp = np.zeros((intrinsics_corners_nb[0] * intrinsics_corners_nb[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:intrinsics_corners_nb[0], 0:intrinsics_corners_nb[1]].T.reshape(-1, 2)
        objp[:, :2] *= intrinsics_square_size

        objpoints = []
        imgpoints = []

        logging.info(f'\nCamera {cam}:')
        img_vid_files = glob.glob(os.path.join(calib_dir, 'intrinsics', cam, f'*.{intrinsics_extension}'))
        if len(img_vid_files) == 0:
            logging.exception(f'The folder {os.path.join(calib_dir, "intrinsics", cam)} does not exist or does not contain any files with extension .{intrinsics_extension}.')
            raise ValueError(f'The folder {os.path.join(calib_dir, "intrinsics", cam)} does not exist or does not contain any files with extension .{intrinsics_extension}.')

        img_vid_files = sorted(img_vid_files, key=lambda c: [int(n) for n in re.findall(r'\d+', c)])

        try:
            cap = cv2.VideoCapture(img_vid_files[0])
            cap.read()
            if cap.read()[0] == False:
                raise
            extract_frames(img_vid_files[0], extract_every_N_sec, overwrite_extraction)
            img_vid_files = glob.glob(os.path.join(calib_dir, 'intrinsics', cam, f'*.png'))
            img_vid_files = sorted(img_vid_files, key=lambda c: [int(n) for n in re.findall(r'\d+', c)])
        except:
            pass

        for img_path in img_vid_files:
            imgp_confirmed = findCorners(img_path, intrinsics_corners_nb, objp=objp, show=show_detection_intrinsics)
            if imgp_confirmed is not None:
                imgpoints.append(imgp_confirmed)
                objpoints.append(objp)

        if len(imgpoints) == 0:
            logging.error(f"No corners detected for camera {cam}. Cannot perform calibration.")
            raise ValueError(f"No corners detected for camera {cam}. Cannot perform calibration.")

        img = cv2.imread(str(img_path))
        if img is None:
            logging.error(f"Failed to read image {img_path}.")
            raise ValueError(f"Failed to read image {img_path}.")

        ret_cam, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None, flags=cv2.CALIB_FIX_K3)

        h, w = [np.float32(i) for i in img.shape[:-1]]
        ret.append(ret_cam)
        C.append(cam)
        S.append([w, h])
        D.append(dist[0])
        K.append(mtx)
        R.append([0.0, 0.0, 0.0])
        T.append([0.0, 0.0, 0.0])

        logging.info(f'Intrinsics error: {np.around(ret_cam, decimals=3)} px for each cameras.')
        print(f'Intrinsics error: {np.around(ret_cam, decimals=3)} px for each cameras {cam}.')
        print(mtx, dist)

    return ret, C, S, D, K, R, T

# Пример использования
calib_dir = '/content/drive/My Drive/image/'
intrinsics_config_dict = {
    'overwrite_intrinsics': False,
    'intrinsics_extension': 'jpg',
    'extract_every_N_sec': 1,
    'show_detection_intrinsics': True,
    'intrinsics_corners_nb': (3, 5),
    'intrinsics_square_size': 100  # В мм
}

try:
    ret, C, S, D, K, R, T = calib_calc_fun(calib_dir,   intrinsics_config_dict)
except Exception as e:
    logging.error(f'An error occurred: {e}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Шахматка не найдена /content/drive/My Drive/image/intrinsics/cam4/frame_0215.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0216.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0217.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0218.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0219.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0220.jpg
Шахматка найдена /content/drive/My Drive/image/intrinsics/cam4/Копия frame_0221.jpg
Intrinsics error: 1.243 px for each cameras cam4.
[[2.22939947e+03 0.00000000e+00 8.74549620e+02]
 [0.00000000e+00 2.97371458e+03 5.12177631e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 0.64678577 -3.90094255 -0.04001222 -0.00876333  0.        ]]
Шахматка не найдена /conten

In [ ]:
#Intrinsics error: 2.322 px for each cameras cam1.
[[1.69907405e+03 0.00000000e+00 1.14283754e+03]
 [0.00000000e+00 1.69396013e+03 3.98611679e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 0.24778605 -0.67165075 -0.03581033  0.0121675   0.        ]]

#Intrinsics error: 3.519 px for each cameras cam2.
[[2.02060603e+03 0.00000000e+00 1.13166384e+03]
 [0.00000000e+00 1.81066168e+03 3.29930972e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[-0.97520958  1.27086966 -0.09989699  0.04843398  0.        ]]

#Intrinsics error: 2.4 px for each cameras cam3.
[[1.59848021e+03 0.00000000e+00 1.11090665e+03]
 [0.00000000e+00 1.59459706e+03 4.54429998e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 0.01369584  0.01547548 -0.01721357  0.02961253  0.        ]]

#Intrinsics error: 1.243 px for each cameras cam4.
[[2.22939947e+03 0.00000000e+00 8.74549620e+02]
 [0.00000000e+00 2.97371458e+03 5.12177631e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 0.64678577 -3.90094255 -0.04001222 -0.00876333  0.        ]]

In [ ]:
#@title Обрезка видео

import os

# Путь к видео
video_paths = {
    'cam1.mp4': '/content/drive/My Drive/video/cam1.mp4',
    'cam2.mp4': '/content/drive/My Drive/video/cam2.mp4',
    'cam3.mp4': '/content/drive/My Drive/video/cam3.mp4',
    'cam4.mp4': '/content/drive/My Drive/video/cam4.mp4'
}

# Временные метки для обрезки (начало и конец в секундах)
cut_times = {
    'cam1.mp4': (144, 186),
    'cam2.mp4': (181, 223),
    'cam3.mp4': (220, 262),
    'cam4.mp4': (259, 301)
}

# Папка для сохранения обрезанных видео
output_folder = '/content/drive/My Drive/video/cut_videos/'

# Создаем папку для обрезанных видео, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Обрезка видео
for video_name, (start, end) in cut_times.items():
    video_path = video_paths[video_name]
    output_video_path = os.path.join(output_folder, f'cut_{video_name}')

    # Вычисляем длительность обрезки
    duration = end - start

    # Команда для обрезки видео
    command = f"ffmpeg -i '{video_path}' -ss {start} -t {duration} -c copy '{output_video_path}'"

    # Выполняем команду
    os.system(command)

print("Обрезка видео завершена.")

Обрезка видео завершена.


In [ ]:
#@title Получение синхронизированного видео с рапознанными позами
import cv2
from moviepy.editor import VideoFileClip, clips_array

# Пути к исходным видеофайлам
cam_01_path = '/content/drive/MyDrive/video/pose/cam1_pose.mp4'
cam_02_path = '/content/drive/MyDrive/video/pose/cam2_pose.mp4'
cam_03_path = '/content/drive/MyDrive/video/pose/cam3_pose.mp4'
cam_04_path = '/content/drive/MyDrive/video/pose/cam4_pose.mp4'

# Загружаем видеофайлы как объекты VideoClip
cam_01 = VideoFileClip(cam_01_path)
cam_02 = VideoFileClip(cam_02_path)
cam_03 = VideoFileClip(cam_03_path)
cam_04 = VideoFileClip(cam_04_path)

# Создаем массив клипов для объединения
final_clip = clips_array([[cam_01, cam_02],
                          [cam_03, cam_04]])

# Делаем все клипы одинаковой длительности (если они имеют разную длину)
final_clip = final_clip.set_duration(min(cam_01.duration, cam_02.duration, cam_03.duration, cam_04.duration))

# Экспорт результата
output_path = "/content/drive/MyDrive/video/pose/final_video_pose.mp4"
final_clip.write_videofile(output_path, codec="libx264", fps=30)  # Укажите желаемый FPS

Полученное видео: https://drive.google.com/file/d/1SXfQPDp6mOxw9YgjmqhiHjqnRIl7CeCd/view?usp=sharing
